# To enable LLM's memory (remembering the context)
remember previous parts of the conversation and feed that into the language model so that they can have this 
conversational flow as you're interacting with them. 

# LangChain: Memory

## Outline
* ConversationBufferMemory : store all messages.
* ConversationBufferWindowMemory : store conversations based on the number of window in parameter setting. eg. when k=1, store only the latest conversation input.
* ConversationTokenBufferMemory : store conversations based on the number of tokens in parameter setting. eg. when max_token_limit=100, store only the latest 100 words from conversation input.
* ConversationSummaryMemory : store the summary of the input text with limited token number.

# OpenAI API key access

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

# ConversationBufferMemory

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
# create model and set the memory type to "ConversationBufferMemory"
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [4]:
# insert a few input msg
conversation.predict(input="Hi, my name is Christine.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Christine.
AI:

> Finished chain.


"Hello Christine! It's nice to meet you. How can I assist you today?"

In [5]:
# insert a few input msg
conversation.predict(input="What is the weather like today in Toronto?")

''' you can see from Current conversation, the conversations are being stored in the memory.'''



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Christine.
AI: Hello Christine! It's nice to meet you. How can I assist you today?
Human: What is the weather like today in Toronto?
AI:

> Finished chain.


"I'm sorry, but I don't have access to real-time data. However, you can easily check the weather in Toronto by using a weather website or app."

In [6]:
# test if the model memorize the information from the last conversation
conversation.predict(input="What is my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Christine.
AI: Hello Christine! It's nice to meet you. How can I assist you today?
Human: What is the weather like today in Toronto?
AI: I'm sorry, but I don't have access to real-time data. However, you can easily check the weather in Toronto by using a weather website or app.
Human: What is my name?
AI:

> Finished chain.


'Your name is Christine.'

In [7]:
# To store input, output manually, use 'save_context'
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [8]:
# To check what content has memory saved
print(memory.buffer)

Human: Hi, my name is Christine.
AI: Hello Christine! It's nice to meet you. How can I assist you today?
Human: What is the weather like today in Toronto?
AI: I'm sorry, but I don't have access to real-time data. However, you can easily check the weather in Toronto by using a weather website or app.
Human: What is my name?
AI: Your name is Christine.
Human: Hi
AI: What's up


In [9]:
# To check what content has memory saved. Can add more specific parameter in the bracket to tune the output.
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Christine.\nAI: Hello Christine! It's nice to meet you. How can I assist you today?\nHuman: What is the weather like today in Toronto?\nAI: I'm sorry, but I don't have access to real-time data. However, you can easily check the weather in Toronto by using a weather website or app.\nHuman: What is my name?\nAI: Your name is Christine.\nHuman: Hi\nAI: What's up"}

# ConversationBufferWindowMemory

In [11]:
from langchain.memory import ConversationBufferWindowMemory

In [12]:
# by setting k=1, the memory saves only the last 1 window conversation
memory = ConversationBufferWindowMemory(k=1)               

In [13]:
# 1 window
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
# 2 window
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [14]:
# check what content has memory saved
memory.load_memory_variables({})

''' it saves only the last 1 window of conversation.'''

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [15]:
# To test with LLM, with ConversationBufferWindowMemory k=1
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

conversation.predict(input="Hi, my name is Tom.")

"Hello Tom! It's nice to meet you. How can I assist you today?"

In [16]:
# random msg 
conversation.predict(input="What is starbucks?")

'Starbucks is a well-known coffeehouse chain that was founded in 1971 in Seattle, Washington. It has since become one of the largest and most recognizable coffeehouse chains in the world, with thousands of locations in various countries. Starbucks is known for its wide range of coffee and tea beverages, as well as its pastries, sandwiches, and other food items. The company also sells whole bean coffee, coffee accessories, and other merchandise. Starbucks is often a popular destination for people looking for a place to relax, work, or socialize.'

In [17]:
#  To test if the model save the memory of what's the user's name, prior to the lastest window of conversation
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."

# ConversationTokenBufferMemory

In [18]:
# !pip install tiktoken

     |████████████████████████████████| 798 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 3.8 MB/s  eta 0:00:01
     |████████████████████████████████| 294 kB 21.6 MB/s eta 0:00:01
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4
    Uninstalling regex-2021.4.4:
      Successfully uninstalled regex-2021.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.


In [19]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [20]:
# set the memory type to ConversationTokenBufferMemory, and limit the token number to only 30 words
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [21]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

# ConversationSummaryMemory

In [22]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})